<a href="https://colab.research.google.com/github/AfekIlayAdler/Uni-2019-Dynamic_Pricing/blob/master/HW1/HW1_Practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Imports

In [1]:
import torch
from torchvision import datasets, transforms
import helper
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
#from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
from tensorflow import summary
import datetime
%load_ext tensorboard

In [2]:
current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)

AttributeError: ignored

## 2. Get data


In [0]:
"""
# TO DO: understand transform
"""
def get_train_test_loaders():
  # Define a transform to normalize the data
  transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
  # Download and load the training data
  trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
  # Download and load the test data
  testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
  testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
  return trainloader, testloader

In [0]:
# import matplotlib.pyplot as plt
# % matplotlib inline
# image, label = next(iter(trainloader))
# print (image.size())
# image = image[2][0]
# plt.imshow(image)

# 3. Model


In [0]:
class LeNet5(nn.Module):
  def __init__(self,regulizer):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5,padding=2)
    self.max_pool_1 = nn.MaxPool2d(kernel_size=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    self.max_pool_2 = nn.MaxPool2d(kernel_size=2)  
    self.fc1 = nn.Linear(16*5*5, 120) 
    self.fc2 = nn.Linear(120, 84) 
    self.fc3 = nn.Linear(84, 10) 
    self.softmax = nn.LogSoftmax(dim=1)
    self.regulizer = regulizer

  def forward(self, x):
    x = F.relu(self.conv1(x))  
    x = self.max_pool_1(x) 
    x = F.relu(self.conv2(x))
    x = self.max_pool_2(x)
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = self.regulizer(x)
    x = self.softmax(x)
    return x

class LeNet5Drop(LeNet5):
  def __init__(self,regulizer = nn.Dropout(0.1)):
    super().__init__(regulizer)

class LeNet5BatchNorm(LeNet5):
    def __init__(self,regulizer = nn.BatchNorm1d(84, affine=False)):
      super().__init__(regulizer)

class LeNet5NoRegulizer(LeNet5):
    def __init__(self,regulizer = lambda x: x):
      super().__init__(regulizer)

In [0]:
def calc_train_accuracy(model,images,labels):
  model.eval()
  log_ps = model(images)
  ps = torch.exp(log_ps)
  top_p, top_class = ps.topk(1, dim=1)
  equals = top_class == labels.view(*top_class.shape)
  model.train()
  return torch.mean(equals.type(torch.FloatTensor))

def fit(model, trainloader, testloader,weight_decay = 0,num_epochs=5): 
  train_accu, test_accu = [],[]
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr=10**(-3), weight_decay = weight_decay)
  for epoch in range(1,num_epochs+1):
    model.train()
    train_loss , train_accuracy= 0, 0
    for images, labels in trainloader:
      log_ps = model(images)
      train_accuracy += calc_train_accuracy(model,images,labels)
      loss = criterion(log_ps, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

    # recording to Tensorboard
    with train_summary_writer.as_default():
      tf.summary.scalar('accuracy', train_accuracy/len(trainloader), step=epoch)

    train_accu.append(train_accuracy/len(trainloader))
    test_accuracy = test(model,testloader,criterion)
    test_accu.append(test_accuracy/len(testloader))

    # recording to Tensorboard
    with train_summary_writer.as_default():
      tf.summary.scalar('accuracy', test_accuracy/len(testloader), step=epoch)
  return train_accuracy/len(trainloader),test_accuracy/len(testloader)
        
def test(model, test_loader,criterion):
  model.eval()
  test_loss,test_accuracy = 0,0
  with torch.no_grad():
      for images, labels in testloader:
          log_ps = model(images)
          test_loss += criterion(log_ps, labels)
          ps = torch.exp(log_ps)
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          test_accuracy += torch.mean(equals.type(torch.FloatTensor))
          return test_accuracy

In [0]:
def run_exp():
  retults = []
  experiments = {'droput' : LeNet5Drop,
                'batch_norm' : LeNet5BatchNorm,
                'no_regulizer' : LeNet5NoRegulizer,
                'weight_decay' : LeNet5}
  for exp_name, model in experiments.items():
    model = model()
    weight_decay = 10^(-2) if exp_name == weight_decay else 0
    train_acc,test_acc = fit(lenet5, trainloader, testloader,weight_decay = weight_decay)
    retults.append([exp_name,train_acc,test_acc])
  df = pd.DataFrame(retults,columns =['Method','Train Accuracy', 'Test Accuracy'])
  df.to_latex(index=False,bold_rows=True)

In [0]:
trainloader, testloader = get_train_test_loaders()
run_exp()